In [1]:
import json

import pandas as pd

from pathlib import Path

from fastai.text.transform import BaseTokenizer, Tokenizer, Vocab
from fastai.text.data import TextLMDataBunch
import swifter
import fastai

import numpy as np

from bpemb import BPEmb

In [2]:
fastai.__version__

'1.0.42'

In [3]:
bpemb_de = BPEmb(lang="de", vs=25000, dim=300)

In [4]:
bpemb_de = BPEmb(lang="de", vs=25000, dim=300)
itos = dict(enumerate(bpemb_de.words + ['xxpad']))
voc = Vocab(itos)

In [5]:
input_p = '/mnt/data/group07/johannes/germanlm/proc/final_merged'
# input_p = '/mnt/data/group07/johannes/germanlm/proc/final_lemma'

In [6]:
tmp = []
for p in list(Path(input_p).glob('*.txt'))[:-3]:
    tmp.append(pd.read_table(p, header=None))
df_train = pd.concat(tmp)
del tmp

In [7]:
df_train[0] = df_train[0].swifter.apply(lambda x: bpemb_de.encode_ids_with_bos_eos(x))

Pandas Apply: 100%|██████████| 2978665/2978665 [1:58:27<00:00, 419.10it/s]  


In [8]:
df_train.to_csv('/mnt/data/group07/johannes/germanlm/proc/tokenized_lemma/df_train.csv')

In [9]:
del df_train

In [10]:
tmp = []
for p in list(Path(input_p).glob('*.txt'))[-3:]:
    tmp.append(pd.read_table(p, header=None))
df_val = pd.concat(tmp)
del tmp

In [11]:
df_val[0] = df_val[0].swifter.apply(lambda x: bpemb_de.encode_ids_with_bos_eos(x))

Pandas Apply: 100%|██████████| 299992/299992 [11:56<00:00, 418.77it/s]


In [12]:
df_val.to_csv('/mnt/data/group07/johannes/germanlm/proc/tokenized_lemma/df_val.csv')

In [10]:
class BpembTokenizer(BaseTokenizer):
    def tokenizer(self, t):
        return bpemb_de.encode(t)
#         res = []
#         doc = nlp(t)
#         for s in docs.sents:
#             res += bpemb_de.encode(s)
#         return res

In [11]:
# class BpembTokenizer(BaseTokenizer):
#     def tokenizer(self, t):
#         res = []
#         tokenizer = somajo.Tokenizer(split_camel_case=True, token_classes=False, extra_info=False)
#         tokens = tokenizer.tokenize(str(t))
        
#         sentence_splitter = somajo.SentenceSplitter(is_tuple=False)
#         sentences = sentence_splitter.split(tokens)

#         for s in sentences:
#             print(s)
#             res += bpemb_de.encode(' '.join(s))
#         return res

In [ ]:
# tok = Tokenizer(tok_func=BpembTokenizer)

In [13]:
train_ids = pd.read_csv('/mnt/data/group07/johannes/germanlm/proc/tokenized_lemma/df_train.csv')

In [14]:
val_ids = pd.read_csv('/mnt/data/group07/johannes/germanlm/proc/tokenized_lemma/df_val.csv')

In [15]:
val_ids = np.array([np.array(json.loads(x[0])) for x in val_ids[['0']].values])
train_ids = np.array([np.array(json.loads(x[0])) for x in train_ids[['0']].values])

In [11]:
# itos = dict(enumerate(bpemb_de.words))

In [12]:
# itos

{0: '<unk>',
 1: '<s>',
 2: '</s>',
 3: 'er',
 4: 'en',
 5: '▁d',
 6: 'ch',
 7: 'in',
 8: '00',
 9: 'ie',
 10: '▁s',
 11: 'un',
 12: '▁a',
 13: 'te',
 14: 'ei',
 15: 'an',
 16: '▁b',
 17: 'es',
 18: 'is',
 19: '▁w',
 20: 'on',
 21: '▁00',
 22: '▁v',
 23: 'ein',
 24: 'or',
 25: '▁m',
 26: '▁der',
 27: '▁g',
 28: '▁k',
 29: '▁un',
 30: 'ar',
 31: '▁die',
 32: 'ur',
 33: '▁f',
 34: 'st',
 35: "''",
 36: '▁0000',
 37: 'al',
 38: 'it',
 39: '▁in',
 40: 'ich',
 41: '▁z',
 42: 'el',
 43: '▁und',
 44: '▁h',
 45: '▁n',
 46: '▁ein',
 47: 'at',
 48: 'us',
 49: 'ung',
 50: 'em',
 51: 'as',
 52: '▁p',
 53: 'ten',
 54: 'sch',
 55: '▁l',
 56: 'ter',
 57: 'de',
 58: 'hr',
 59: '▁(',
 60: '▁er',
 61: '▁r',
 62: 'ig',
 63: 'isch',
 64: '▁i',
 65: 'ol',
 66: 'il',
 67: '▁*',
 68: 'der',
 69: 'den',
 70: '▁au',
 71: '▁j',
 72: '▁t',
 73: 'am',
 74: 'ber',
 75: 'au',
 76: '▁von',
 77: 'ge',
 78: "▁''",
 79: '▁an',
 80: 'ach',
 81: '▁ver',
 82: 'et',
 83: 'um',
 84: 'gen',
 85: '▁al',
 86: '▁st',
 87: 'and'

In [13]:
# itos[0] = 'xxunk'

In [14]:
# itos[1] = 'xxpad'

In [15]:
# voc = Vocab(itos)

In [16]:
data = TextLMDataBunch.from_ids(path='/mnt/data/group07/johannes/germanlm/exp_10', vocab=voc, train_ids=train_ids, valid_ids=val_ids)

In [17]:
data

TextLMDataBunch;

Train: LabelList
y: LMLabel (2978665 items)
[Category 0, Category 0, Category 0, Category 0, Category 0]...
Path: /mnt/data/group07/johannes/germanlm/exp_10
x: LMTextList (2978665 items)
[Text <s> ▁james ▁andrew ▁be all ▁james ▁andrew ▁be all ▁(* ▁00. ▁oktober ▁0000 ▁bei ▁mid l oth ian , ▁texas ; ▁+ ▁00. ▁februar ▁0000 ▁in ▁dallas , ▁texas ) ▁war ▁ein ▁us - amerikanischer ▁politiker . ▁zwischen ▁0000 ▁und ▁0000 ▁vertrat ▁er ▁den ▁bundesstaat ▁texas ▁im ▁us - repräsentanten haus . ▁werde gang . ▁james ▁be all ▁besuchte ▁die ▁öffentlichen ▁schulen ▁seiner ▁heimat . ▁in ▁den ▁jahren ▁0000 ▁und ▁0000 ▁unterrichtete ▁er ▁selbst ▁als ▁lehrer . ▁nach ▁einem ▁anschließenden ▁jur ast udium ▁an ▁der ▁university ▁of ▁texas ▁in ▁austin ▁und ▁seiner ▁0000 ▁erfolgten ▁zulassung ▁als ▁rechtsanwalt ▁begann ▁er ▁im ▁ell is ▁county ▁in ▁diesem ▁beruf ▁zu ▁arbeiten . ▁gleichzeitig ▁schlug ▁er ▁als ▁mitglied ▁der ▁demokratischen ▁partei ▁eine ▁politische ▁laufbahn ▁ein . ▁von ▁0000 ▁bis 

In [18]:
data.save()